# ELS Pricer - GPU Benchmark on Google Colab

**C++/CUDA 기반 2자산 Step-Down ELS Pricer 성능 테스트**

이 노트북은 다음을 테스트합니다:
- ✅ CPU 버전 (6가지 그리드 크기)
- ✅ GPU 버전 (6가지 그리드 크기)
- ✅ CPU vs GPU 성능 비교
- ✅ 버그 수정된 정확한 가격 계산

## 🔧 최근 업데이트 (2024-11-14)
- **조기상환 버그 수정**: 강제 상환으로 변경 (이전: 선택적)
- **타임스텝 버그 수정**: 만기 관측일 포함
- **가격 정확도 향상**: 111.7원 → 103.9원 (오차 7% → 0.5%)

## 📋 테스트 그리드 크기

| # | N1 | N2 | Nt | 총 Points | 설명 |
|---|----|----|----|-----------| |
| 1 | 100 | 100 | 200 | 2M | Small - Medium density |
| 2 | 100 | 100 | 1000 | 10M | Small - High time resolution |
| 3 | 200 | 200 | 200 | 8M | Large - Medium density |
| 4 | 200 | 200 | 1000 | 40M | Large - High time resolution |
| 5 | 400 | 400 | 200 | 32M | Very Large - Medium density |
| 6 | 400 | 400 | 1000 | 160M | Very Large - High time resolution |

---

## Step 1: GPU 환경 확인

In [ ]:
# GPU 확인
!nvidia-smi
print("\n" + "="*70)
!nvcc --version

## Step 2: 필요한 도구 설치

In [ ]:
# CMake와 빌드 도구 설치
!apt-get update -qq
!apt-get install -y -qq cmake build-essential

# 버전 확인
!cmake --version
!g++ --version | head -1

## Step 3: 프로젝트 업로드

**방법 1: 파일 직접 업로드 (추천)**
- `els-pricer-colab.tar.gz` 파일을 준비 (버그 수정 버전)
- 아래 셀 실행 후 파일 선택

**방법 2: Google Drive 마운트**
- Drive에 미리 업로드한 경우

**📌 중요**: 2024-11-14 이후 생성된 `els-pricer-colab.tar.gz` 파일을 사용하세요 (버그 수정 포함)

In [ ]:
# 방법 1: 파일 업로드
from google.colab import files
import os

print("📁 els-pricer-colab.tar.gz 파일을 업로드해주세요...")
print("   (버그 수정 버전: 2024-11-14 이후 생성)")
uploaded = files.upload()

# 압축 해제
if 'els-pricer-colab.tar.gz' in uploaded:
    !tar -xzf els-pricer-colab.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료!")
    !ls -la els-pricer-cpp/
elif 'els-pricer-cpp.tar.gz' in uploaded:
    # 구버전 파일명도 지원
    !tar -xzf els-pricer-cpp.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료! (구버전 파일명)")
    !ls -la els-pricer-cpp/
else:
    print("❌ els-pricer-colab.tar.gz 파일을 찾을 수 없습니다.")

In [ ]:
# (선택사항) 방법 2: Google Drive에서 가져오기
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/els-pricer-colab.tar.gz .
# !tar -xzf els-pricer-colab.tar.gz

## Step 4: 프로젝트 빌드 (GPU 버전)

In [ ]:
import os

# 프로젝트 디렉토리 확인
project_dir = '/content/els-pricer-cpp'
if not os.path.exists(project_dir):
    print("❌ 프로젝트 디렉토리를 찾을 수 없습니다!")
    print("   Step 3에서 파일을 업로드하고 압축을 해제했는지 확인하세요.")
else:
    print("✓ 프로젝트 디렉토리 확인됨")
    
    # 작업 디렉토리 변경
    os.chdir(project_dir)
    print(f"✓ 작업 디렉토리: {os.getcwd()}")
    
    # 빌드 디렉토리 생성
    !rm -rf build
    !mkdir -p build
    
    # 빌드 디렉토리로 이동
    os.chdir(os.path.join(project_dir, 'build'))
    print(f"✓ 빌드 디렉토리: {os.getcwd()}")
    
    # CMake 설정 (GPU 포함)
    print("\n🔧 CMake 설정 중...\n")
    !cmake ..
    
    print("\n" + "="*70)
    print("🔨 컴파일 중 (병렬 빌드)...\n")
    !make -j4
    
    print("\n" + "="*70)
    print("✓ 빌드 완료!")
    !ls -lh els_pricer test_els benchmark_cpu_vs_gpu validate_price 2>/dev/null || echo "일부 실행 파일이 없을 수 있습니다."

## Step 5: CPU vs GPU 벤치마크 실행

빌드가 완료되었습니다! 이제 CPU와 GPU 성능을 직접 비교하는 벤치마크를 실행합니다.

**테스트할 그리드:**
- 100×100×200, 100×100×1000
- 200×200×200, 200×200×1000
- 400×400×200, 400×400×1000

각 그리드마다 CPU와 GPU 모두에서 실행하고 어느 쪽이 더 빠른지 확인합니다.

In [ ]:
import os

# 빌드 디렉토리로 이동
build_dir = '/content/els-pricer-cpp/build'
if os.path.exists(build_dir):
    os.chdir(build_dir)
    print(f"✓ 작업 디렉토리: {os.getcwd()}")
    
    print("\n🚀 CPU vs GPU 종합 벤치마크 시작!\n")
    print("="*70 + "\n")
    
    # CPU vs GPU 비교 프로그램 실행 (6개 그리드 모두 테스트)
    if os.path.exists('./benchmark_cpu_vs_gpu'):
        !./benchmark_cpu_vs_gpu
    else:
        print("⚠️ benchmark_cpu_vs_gpu 실행 파일을 찾을 수 없습니다.")
        print("   대신 els_pricer로 간단한 테스트를 실행합니다.")
        if os.path.exists('./els_pricer'):
            !./els_pricer
        else:
            print("❌ 실행 파일이 없습니다. 빌드가 성공적으로 완료되었는지 확인하세요.")
else:
    print("❌ 빌드 디렉토리를 찾을 수 없습니다. Step 4를 먼저 실행하세요.")

## Step 6: 결과 데이터 확인 및 분석

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 파일 경로
csv_path = '/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv'

if os.path.exists(csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    
    print("📊 CPU vs GPU 벤치마크 결과:\n")
    print(df.to_string(index=False))
    
    print("\n" + "═"*70)
    print("📈 통계 요약:")
    print(f"   평균 CPU 시간: {df['CPU_Time'].mean():.3f}초")
    print(f"   평균 GPU 시간: {df['GPU_Time'].mean():.3f}초")
    print(f"   평균 가속 비율: {df['Speedup'].mean():.2f}×")
    print(f"   최대 가속 비율: {df['Speedup'].max():.2f}×")
    print(f"   GPU 승리 횟수: {(df['Speedup'] > 1.0).sum()}/6")
    print(f"   CPU 승리 횟수: {(df['Speedup'] < 1.0).sum()}/6")
    
    # 각 그리드별 승자 표시
    print("\n🏆 그리드별 승자:")
    for idx, row in df.iterrows():
        grid_name = f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}"
        if row['Speedup'] > 1.0:
            winner = f"GPU ✓ ({row['Speedup']:.2f}× 빠름)"
        elif row['Speedup'] < 1.0:
            winner = f"CPU ✓ ({1.0/row['Speedup']:.2f}× 빠름)"
        else:
            winner = "무승부"
        print(f"   {grid_name:16s} → {winner}")
else:
    print("⚠️ 벤치마크 결과 CSV 파일을 찾을 수 없습니다.")
    print("   Step 5에서 벤치마크를 먼저 실행하세요.")

## Step 7: 성능 비교 시각화

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path = '/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    
    # 그리드 라벨 생성
    grid_labels = [f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}" 
                   for _, row in df.iterrows()]
    
    # 플롯 생성
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('ELS Pricer: CPU vs GPU Performance Comparison', fontsize=16, fontweight='bold')
    
    # 1. CPU vs GPU 실행 시간 비교
    ax1 = axes[0, 0]
    x = np.arange(len(grid_labels))
    width = 0.35
    bars1 = ax1.bar(x - width/2, df['CPU_Time'], width, label='CPU', color='steelblue', alpha=0.8)
    bars2 = ax1.bar(x + width/2, df['GPU_Time'], width, label='GPU', color='coral', alpha=0.8)
    ax1.set_xlabel('Grid Size', fontweight='bold')
    ax1.set_ylabel('Time (seconds)', fontweight='bold')
    ax1.set_title('Execution Time Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. 가속 비율
    ax2 = axes[0, 1]
    colors = ['green' if s > 1.0 else 'red' for s in df['Speedup']]
    bars = ax2.bar(grid_labels, df['Speedup'], color=colors, alpha=0.7)
    ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even')
    ax2.set_xlabel('Grid Size', fontweight='bold')
    ax2.set_ylabel('Speedup (×)', fontweight='bold')
    ax2.set_title('GPU Speedup over CPU')
    ax2.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # 가속비 값 표시
    for i, (label, speedup) in enumerate(zip(grid_labels, df['Speedup'])):
        ax2.text(i, speedup + 0.5, f'{speedup:.1f}×', 
                 ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # 3. 가격 수렴성
    ax3 = axes[1, 0]
    ax3.plot(grid_labels, df['CPU_Price'], 'o-', label='CPU Price', markersize=8, linewidth=2)
    ax3.plot(grid_labels, df['GPU_Price'], 's-', label='GPU Price', markersize=8, linewidth=2)
    ax3.set_xlabel('Grid Size', fontweight='bold')
    ax3.set_ylabel('ELS Price', fontweight='bold')
    ax3.set_title('Price Convergence')
    ax3.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Points/second 처리 속도
    ax4 = axes[1, 1]
    cpu_throughput = df['Total_Points'] / df['CPU_Time'] / 1e6
    gpu_throughput = df['Total_Points'] / df['GPU_Time'] / 1e6
    x = np.arange(len(grid_labels))
    bars1 = ax4.bar(x - width/2, cpu_throughput, width, label='CPU', color='steelblue', alpha=0.8)
    bars2 = ax4.bar(x + width/2, gpu_throughput, width, label='GPU', color='coral', alpha=0.8)
    ax4.set_xlabel('Grid Size', fontweight='bold')
    ax4.set_ylabel('Throughput (M points/sec)', fontweight='bold')
    ax4.set_title('Processing Throughput')
    ax4.set_xticks(x)
    ax4.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('cpu_vs_gpu_benchmark.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ 그래프가 cpu_vs_gpu_benchmark.png로 저장되었습니다!")
else:
    print("⚠️ 시각화할 데이터가 없습니다. Step 5를 먼저 실행하세요.")

## Step 8: 결과 파일 다운로드

In [ ]:
from google.colab import files
import os

# 다운로드할 파일 목록
files_to_download = [
    ('/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv', 'csv'),
    ('cpu_vs_gpu_benchmark.png', 'png')
]

downloaded = []
not_found = []

for file_path, file_type in files_to_download:
    if os.path.exists(file_path):
        files.download(file_path)
        downloaded.append(file_path)
    else:
        not_found.append(file_path)

if downloaded:
    print("✅ 다운로드 완료:")
    for f in downloaded:
        print(f"  - {os.path.basename(f)}")

if not_found:
    print("\n⚠️ 파일을 찾을 수 없음:")
    for f in not_found:
        print(f"  - {os.path.basename(f)}")
    print("  Step 5-7을 먼저 실행하세요.")

## Step 9: 가격 검증 테스트

실제 시장 파라미터로 ELS 가격을 계산하여 검증합니다.

**테스트 조건:**
- 기초자산 1: 변동성 15.2%, 배당 1.5%
- 기초자산 2: 변동성 40.4%, 배당 2.0%
- 상관계수: 0.61
- 무위험 이자율: 3.477%
- 만기: 3년 (6개월 간격 조기상환)
- 조기상환 베리어: 85%, 85%, 80%, 80%, 75%, 70%
- 낙인 베리어: 45%

**⚠️ 버그 수정 완료 (2024-11-14):**
- ✅ 조기상환이 강제(mandatory)로 수정됨 (이전: optional)
- ✅ 타임스텝 루프가 만기 관측일을 포함하도록 수정됨
- **예상 가격**: ~103.9원 (KI 미추적), ~93.9원 (KI 추적시)

In [ ]:
import os

build_dir = '/content/els-pricer-cpp/build'
if os.path.exists(build_dir):
    os.chdir(build_dir)
    
    print("🔍 Running Price Validation Test...\n")
    print("="*70 + "\n")
    
    if os.path.exists('./validate_price'):
        !./validate_price
    elif os.path.exists('./validate_price_cpu'):
        !./validate_price_cpu
    else:
        print("⚠️ validate_price 실행 파일을 찾을 수 없습니다.")
        print("   빌드가 완료되었는지 확인하세요.")
    
    print("\n" + "="*70)
else:
    print("❌ 빌드 디렉토리를 찾을 수 없습니다. Step 4를 먼저 실행하세요.")

## 📌 요약

### 테스트 완료!

이 노트북에서:
1. ✅ GPU 환경 확인 (nvidia-smi, nvcc)
2. ✅ C++/CUDA 프로젝트 빌드 (CMake)
3. ✅ CPU vs GPU 종합 벤치마크 (6 grids)
4. ✅ 성능 비교 및 각 그리드별 승자 확인
5. ✅ 결과 시각화 및 다운로드

### 버그 수정 사항 (2024-11-14)
- 조기상환 로직 수정: 111.7원 → 103.9원
- 오차 개선: 7% → 0.5%

### 성능 (예상)
- **C++ CPU**: Python 대비 ~90-200× 빠름
- **C++ GPU**: CPU 대비 ~10-15× 추가 가속
- **총 가속**: Python 대비 ~900-3000×

### 결과 파일
- `cpu_vs_gpu_results.csv`: 전체 벤치마크 데이터
- `cpu_vs_gpu_benchmark.png`: 성능 비교 시각화

---

**프로젝트**: ELS Pricer C++/CUDA  
**날짜**: 2024-11-14